# spannerlog Magic

In [ ]:
#| default_exp magic

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from IPython.display import display, HTML
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from typing import Optional
import logging
logger = logging.getLogger(__name__)

from pathlib import Path
import pandas as pd
from IPython.core.magic import (Magics, magics_class, line_cell_magic)
from singleton_decorator import singleton
from spannerlib.utils import assert_df_equals
from spannerlib.session import Session

[autoreload of spannerlib.span failed: Traceback (most recent call last):
  File "/Users/dean/miniconda3/envs/span/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/dean/miniconda3/envs/span/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/Users/dean/miniconda3/envs/span/lib/python3.11/importlib/__init__.py", line 159, in reload
    raise ImportError(msg.format(parent_name),
ImportError: parent 'spannerlib' not in sys.modules
]


In [ ]:
#| export
@singleton
class _MagicSession():
    def __init__(self):
        self.session = Session()

def get_magic_session():
    """Returns the session used by the magic system

    Returns:
        Session
    """    
    return _MagicSession().session

def set_magic_session(session: Session):
    """Changes the session used by the magic system to the one provided

    Args:
        session (Session): the session to use in the magic system
    """
    _MagicSession().session = session


In [ ]:
sess_1 = Session()
sess_1

<spannerlib.session.Session>

In [ ]:
assert get_magic_session()==get_magic_session()
assert get_magic_session()!=sess_1
set_magic_session(sess_1)
assert get_magic_session()==sess_1

#TODO from here add to parser, if we have curly braces try to get var name from global

In [ ]:
#| export
import argparse
import shlex
magic_args_parser = parser = argparse.ArgumentParser()
parser.add_argument("-o", "--output",type=str, help="write code to file")
parser.add_argument("-a", "--append",type=str, help="append code to file")

def curly_brace_to_glob_var(line:str):
    """If the line is a variable name enclosed in curly braces, return the variable from the global scope
    """
    if line.startswith("{") and line.endswith("}"):
        var = line[1:-1]
    else:
        return line

    if var in globals():
        return globals()[var]
    else:
        raise ValueError(f"Variable {var} not found in global scope")

def parse_options(line):
    namespace = magic_args_parser.parse_args(shlex.split(line))
    args_dict = namespace.__dict__
    args_dict = {k:v for k,v in args_dict.items() if v is not None}
    args_dict = {k:curly_brace_to_glob_var(v) for k,v in args_dict.items()}
    return args_dict


In [ ]:
test = "heyyyy.txt"
r = parse_options("-o test, -a {test}")
assert r == {'output': 'test,', 'append': 'heyyyy.txt'}

r = parse_options("-o test")
assert r == {'output': 'test'}

In [ ]:
#| export
def clean_query_lines(code)->str:
    """remove query lines from code by removing lines that start with '?' """
    code_lines = code.split('\n')
    non_query_lines = [line for line in code_lines if not line.startswith('?')]

    # remove double empty lines
    clean_code = []
    for i, line in enumerate(non_query_lines):
        if i>0 and not line and not non_query_lines[i-1]:
            continue
        clean_code.append(line)
    clean_code = '\n'.join(clean_code)
    return clean_code

@magics_class
class spannerlogMagic(Magics):
    @line_cell_magic
    def spannerlog(self, line: str, cell: str) -> None:
        # import locally to prevent circular import issues
        magic_session = get_magic_session()

        args = parse_options(line)

        code = cell if cell else line
        try:
            _ = magic_session.export(code,display_results=True
                # ,return_statements_meta=True
                )
        except Exception as e:
            raise e from None

        if 'output' in args or 'append' in args:
            clean_code = clean_query_lines(code)

        if 'output' in args:
            out_file = Path(args['output'])
            out_file.write_text(clean_code + '\n')
            
        if 'append' in args:
            out_file = Path(args['append'])
            if not clean_code in out_file.read_text():
                with open(out_file, "a") as f:
                    f.write(clean_code + '\n')

def load_ipython_extension(ipython):
    ipython.register_magics(spannerlogMagic)

In [ ]:
try:
    load_ipython_extension(get_ipython())
    logger.warning("spannerlog magic loaded")
except (AttributeError, ImportError):
    pass

spannerlog magic loaded


In [ ]:
%%spannerlog
new A(int)
A(1)
?A(X)

'?A(X)'

,X
0,1


In [ ]:
sesssion = get_magic_session()
assert_df_equals(session.export("?A(X)"),pd.DataFrame({'X':[1]}))

,X
0,1


In [ ]:
%%spannerlog -o text.slog
new A(int)
?A(X)

'?A(X)'

,X
0,1


In [ ]:
%%spannerlog -a text.slog
new B(int)

In [ ]:
%%spannerlog -a text.slog
new B(int)

In [ ]:
assert Path('text.slog').read_text() == 'new A(int)\n\nnew B(int)\n\n'
Path('text.slog').unlink()

In [ ]:
write_to_file = 'text2.slog'

In [ ]:
%%spannerlog -o {write_to_file}
new A(int)

In [ ]:
%%spannerlog -a {write_to_file}
new B(int)

In [ ]:
assert Path(write_to_file).read_text() == 'new A(int)\n\nnew B(int)\n\n'
Path(write_to_file).unlink()

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     